<http://www.atmarkit.co.jp/ait/articles/1312/18/news117.html>

# ITエンジニアのためのデータサイエンティスト養成講座（7）
回帰分析II：重回帰分析の方法、科学的な将来予測


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
matplotlib.style.use('ggplot')

前回は回帰分析にういて、最小二乗法やモデルの評価、妥当性検証などの基本的なことを紹介。

今回は説明変数が2つ以上のより複雑な重回帰分析。

### 再び電力消費量と気温との相関関係


In [6]:
# 東京電力の電力使用状況のデータ（2013年)
# CSVデータ取得
df_power = pd.read_csv('http://www.tepco.co.jp/forecast/html/images/juyo-2013.csv',
                       skiprows=3, names=['date', 'time', 'actual'], encoding='Shift_JIS')
df_power.head()

,date,time,actual
0,2013/1/1,0:00,2873
1,2013/1/1,1:00,2716
2,2013/1/1,2:00,2592
3,2013/1/1,3:00,2482
4,2013/1/1,4:00,2412


In [7]:
# 時系列インデックスの作成
idx_power = pd.to_datetime(df_power['date'] + ' ' + df_power['time'])
df_power.index = idx_power #データフレームのindexに設定

# 不要列の削除
del df_power['date']
del df_power['time']

#電力消費量の日次ダウンサンプリング（日次最大値)
df_power_daily = df_power.resample('D', how='max', kind='period')
df_power_daily.head()

,actual
2013-01-01,3132
2013-01-02,3106
2013-01-03,3558
2013-01-04,4016
2013-01-05,4089


In [16]:
# 気象庁提供の東京都の最高気温、最低気温のデータを取り込み
# http://www.data.jma.go.jp/gmd/risk/obsdl/index.php 
# UIのあるページなのであらかじめダウンロードしたものを使用
df_temp = pd.read_csv('data/tokyo-temp-2013.csv', skiprows=6,
                      names=['date', 'high', None, None,'low', None, None], 
                      usecols=['date', 'high', 'low'],
                      parse_dates=['date'],
                      index_col=['date'],
                      encoding='Shift_JIS')


In [17]:
# 5月から9月までの平日のデータに絞り込み
dfx = df_temp.ix['2013-05-01':'2013-09-30'].resample('B', how='max', kind='period')
dfy = df_power_daily.ix['2013-05-01':'2013-09-30'].resample('B', how='max', kind='period')